In [1]:
%reset -f
import itertools
import numpy as np
%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import networkx as nx

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [2]:
# parameters
solveLPOnly = True # set False to solve as an IP

In [3]:
# K := 1

Supplies= {
    # node i : [supply commodoty[1] ... supply commoditiy[K]],
    1: [12.],
    2: [6.],
    3: [-2],
    4: [0.],
    5: [-9.],
    6: [-7.]}

CapacityCosts = {
    # arc (i,j): [capacity, cost commodity[1] ... cost commodity[K]],
    (1,2): [6., 2],
    (1,3): [8., -5],
    (2,4): [5., 3],
    (2,5): [7., 12],
    (3,5): [5., -5],
    (4,5): [8., 2],
    (4,6): [5., 0],
    (5,6): [5., 4]}

In [4]:
Nodes = list(Supplies.keys()) # get node list from supp data

K = len(Supplies[Nodes[0]]) # get num of commods from supp data

Commods = list(range(1,K+1)) # name the commodities 1,2,...,K

Arcs = list(CapacityCosts.keys()) # get arc list from Capacity/Cost data

ArcsCrossCommods = list(itertools.product(Arcs,Commods)) # make cross product 
# of Arcs and Commods for var indexing

In [9]:
model = gp.Model()

if solveLPOnly == True:
    x = model.addVars(ArcsCrossCommods)
else:
    x = model.addVars(ArcsCrossCommods, vtype = GRB.INTEGER)
model.setObjective(sum(sum(CapacityCosts[i,j][k] * x[(i,j),k] for (i,j) in Arcs) for k in Commods), GRB.MINIMIZE)

model.addConstrs(
    (sum(x[(i,j),k] for j in Nodes if (i,j) in Arcs) - sum(x[(j,i),k] for j in Nodes if (j,i) in Arcs) == Supplies[i][k-1] for i in Nodes for k in Commods))

model.update()

Restricted license - for non-production use only - expires 2025-11-24


In [201]:
model.optimize()
if model.status != GRB.Status.OPTIMAL:
    print("***** Gurobi solve status:", model.status)
    print("***** This is a problem.Model does not have an optimal solution")
    raise StopExecution
print(" ")
print("***** Flows: ")
for (i,j) in Arcs:
    arcflow = ""
    for k in Commods:
        arcflow += str(round(x[(i,j),k].X, 4))
        arcflow += " "
    print("x[(", i, ",", j, "), *]=", arcflow, "capacity:", CapacityCosts[i,j][0])

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x01bb9499
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+01]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -9.2000000e+01   2.000000e+00   0.000000e+00      0s
       1   -8.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective -8.800000000e+01
 
***** Flows: 
x[( 1 , 2 ), *]= 0.0  capacity: 6.0
x[( 1 , 3 ), *]= 12.0  capacity: 8.0
x[( 2 , 4 ), *]= 6.0  capacity: 5.0
x[( 2 , 5 ), *]= 0.0  capa

In [203]:
'''
G = nx.DiGraph()
G.add_nodes_from(Nodes)
G.add_edges_from(Arcs)
plt.figure(figsize=(8,8))
edge_labels = nx.draw_networkx_edge_labels(G, edge_labels = CapacityCosts, pos = nx.shell_layout(G), label_pos=0.3, font_size=10)
nx.draw_shell(G, with_labels=True, node_color='cyan', node_size=800, font_size=20, arrowsize=20)
print("Network with node labels and capacities/costs on arcs")
'''

'\nG = nx.DiGraph()\nG.add_nodes_from(Nodes)\nG.add_edges_from(Arcs)\nplt.figure(figsize=(8,8))\nedge_labels = nx.draw_networkx_edge_labels(G, edge_labels = CapacityCosts, pos = nx.shell_layout(G), label_pos=0.3, font_size=10)\nnx.draw_shell(G, with_labels=True, node_color=\'cyan\', node_size=800, font_size=20, arrowsize=20)\nprint("Network with node labels and capacities/costs on arcs")\n'